In [2]:
import sys
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
x = torch.randn(2,2)
y = torch.randn(2,2)
x, y

(tensor([[-1.1618, -1.5055],
         [ 0.4743,  0.1000]]),
 tensor([[-1.3834,  0.8644],
         [ 0.5144,  0.0782]]))

In [10]:
x[:,1]

tensor([-1.5055,  0.1000])

In [21]:
torch.multiply(x, y)

(tensor([[ 0.3200, -2.4219],
         [-0.0705, -0.3499]]),
 0.32002588)

In [11]:
padded = F.pad(x,(1,1,1,1),mode='constant',value=0)
padded.size()

torch.Size([1, 2, 6, 6])

In [13]:
padded.shape

torch.Size([1, 2, 6, 6])

In [16]:
assert padded.size() == (1,2,6,7)

AssertionError: Padding incorrect

In [11]:
def MyFConv2D(input, weight, bias=None, stride=1, padding=0):
    
    """
    My custom Convolution 2D calculation.

    [input]
    * input    : (batch_size, in_channels, input_height, input_width)
    * weight   : (you have to derive the shape :-)
    * bias     : bias term
    * stride   : stride size
    * padding  : padding size

    [output]
    * output   : (batch_size, out_channels, output_height, output_width)
    """

    assert len(input.shape) == len(weight.shape) , "weight shape incorrect"
    assert len(input.shape) == 4, "input shape incorrect"
    
    k = weight.shape[-1]
    
    # batch_size, in_channels, input_height, input_width = input.shape
    N, C_in, H_in, W_in = input.shape
    # weight shape  (C_out, C_in, kH, kW)
    C_out, C_in, kH, kW = weight.shape 
    b, s, p = bias, stride, padding

    x_pad =  F.pad(input,(p,p,p,p),mode='constant',value=0) # add 2p to last two dims of input
    assert x_pad.shape == (N, C_in, H_in + 2*p, W_in + 2*p) , "padding incorrect"

    # k = 0 # kernel size
    H_out = int( (H_in + 2*p - kH) / s) + 1
    W_out = int( (W_in + 2*p - kW) / s) + 1
    
    ## Derive the output size
    ## Create the output tensor and initialize it with 0
    output = torch.zeros(N, C_out, H_out, W_out)
    
    ## Convolution process
    for b in range(N):
        for c_o in range(C_out):
            for h in range(H_out):
                for w in range(W_out):
                    # get the window
                    window = x_pad[b, :, h*s:h*s+kH, w*s:w*s+kW]
                    # apply the kernel: element-wise multiplication
                    # (b,c_in, kH, KW) * (c_o, c_i, kH, kW) -> (b, c_o, kH, kW)
                    output[b, c_o, h, w] = torch.sum(window * weight[c_o,:,:,:])
                    if bias is not None:
                        output[b, c_o, h, w] += bias[c_o]
    return output

In [12]:
# Create inputs for validation
batch_size, in_channels, H, W = 1, 3, 5, 5  # Example dimensions
out_channels = 2
kernel_size = 3
stride = 1
padding = 1

# Input and weights
input_tensor = torch.rand(batch_size, in_channels, H, W)
weights = torch.rand(out_channels, in_channels, kernel_size, kernel_size)
bias = torch.rand(out_channels)

In [13]:
custom_conv_output = MyFConv2D(input_tensor, weights, bias, stride, padding)
custom_conv_output

tensor([[[[3.6337, 5.5385, 6.2107, 5.8576, 2.7936],
          [4.5595, 7.3083, 7.9239, 9.0680, 4.7402],
          [4.7501, 5.8573, 6.5903, 6.8003, 4.6322],
          [5.0260, 6.2258, 6.5572, 7.0312, 4.8152],
          [2.9814, 4.2286, 4.1540, 3.8835, 2.8146]],

         [[3.6522, 6.6804, 5.6448, 6.1156, 4.0989],
          [5.1535, 7.9776, 7.5715, 8.2140, 5.4167],
          [4.9752, 5.7482, 7.6578, 6.8342, 4.7245],
          [5.0539, 5.4425, 7.1029, 6.7609, 5.3278],
          [3.4588, 4.8966, 4.3990, 4.2119, 3.2847]]]])

In [14]:
# PyTorch Conv2D for comparison
conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
conv2d.weight = torch.nn.Parameter(weights)
conv2d.bias = torch.nn.Parameter(bias)

In [15]:
# Apply PyTorch Conv2D
with torch.no_grad():  # Ensure Conv2D weight and bias are not updated
    torch_conv_output = conv2d(input_tensor)

# Compare outputs
print("Custom Conv2D Output Shape:", custom_conv_output.shape)
print("PyTorch Conv2D Output Shape:", torch_conv_output.shape)
print("Output Difference:", torch.sum(torch_conv_output - custom_conv_output))

Custom Conv2D Output Shape: torch.Size([1, 2, 5, 5])
PyTorch Conv2D Output Shape: torch.Size([1, 2, 5, 5])
Output Difference: tensor(5.9605e-06)
